In [95]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import os
from dotenv import load_dotenv

from datetime import datetime, timezone

In [80]:
STEAM_APP_ID = 812140  # Example: Assassin's Creed Odyssey

#### Loading data from ITAD API

In [81]:
load_dotenv()
ITAD_API_KEY = os.getenv('ITAD_API_KEY')
if not ITAD_API_KEY:
    raise ValueError("Please set the ITAD_API_KEY environment variable.")

In [82]:
url_appID = "https://api.isthereanydeal.com/lookup/id/shop/61/v1"

payload = [f"app/{STEAM_APP_ID}"] # this API accepts a list of identifiers

headers = {
    "Authorization": f"key {ITAD_API_KEY}",
    "Content-Type": "application/json"
}

In [83]:
try:
    resp = requests.post(url_appID, headers=headers, json=payload, timeout=15)
    resp.raise_for_status()
except requests.exceptions.HTTPError as http_err:
    print("HTTP error:", http_err, "| Body:", getattr(resp, "text", ""))
except requests.exceptions.RequestException as req_err:
    print("Request error:", req_err)
else:
    game_id = resp.json()
    print("Raw JSON:", resp.json())

Raw JSON: {'app/812140': '018d937f-0184-7248-8d64-3c723c523111'}


In [84]:
ITAD_GAME_ID = game_id['app/812140']
print("ITAD Game ID:", ITAD_GAME_ID)

ITAD Game ID: 018d937f-0184-7248-8d64-3c723c523111


In [85]:
url_info = f"https://api.isthereanydeal.com/games/info/v2"

params2 = {"id": ITAD_GAME_ID, "key": ITAD_API_KEY}

In [86]:
try:
    r = requests.get(url_info, params=params2, timeout=15)
    r.raise_for_status()
    info = r.json()
    print("Game title:", info.get("title"))
    print("Tags:", info.get("tags"))
    print("Release date:", info.get("releaseDate"))
    print("Publisher:", info.get("publishers"))
    print("Players:", info.get("players"))
    print("Reviews:", info.get("reviews"))
except requests.HTTPError as e:
    print("HTTP error:", e, "| Body:", r.text)

Game title: Assassin's Creed Odyssey
Tags: ['Open World', 'RPG', 'Singleplayer', 'Historical', 'Action']
Release date: 2018-10-03
Publisher: [{'id': 61, 'name': 'Ubisoft'}]
Players: {'recent': 9400, 'day': 10837, 'week': 10837, 'peak': 61984}
Reviews: [{'score': 89, 'source': 'Steam', 'count': 158914, 'url': 'https://store.steampowered.com/app/812140/'}, {'score': 86, 'source': 'Metascore', 'count': 14, 'url': 'https://metacritic.com/game/assassins-creed-odyssey/critic-reviews/?platform=pc'}, {'score': 69, 'source': 'Metacritic User Score', 'count': 5304, 'url': 'https://metacritic.com/game/assassins-creed-odyssey/user-reviews/?platform=pc'}, {'score': 84, 'source': 'OpenCritic', 'count': 169, 'url': 'https://opencritic.com/game/6222/assassins-creed-odyssey'}]


In [91]:
url_history = "https://api.isthereanydeal.com/games/history/v2"

params = {
    "key": ITAD_API_KEY,
    "id": ITAD_GAME_ID,
    "country": "us",
    "shops": "61",
    "since": "2018-01-01T00:00:00Z"
}

In [92]:
try:
    r = requests.get(url_history, params=params, timeout=15)
    print("Request URL:", r.url)
    r.raise_for_status()
    history = r.json()
    print("Price history data:", history)
except requests.HTTPError as e:
    print("HTTP error:", e, "| Body:", r.text)
except requests.RequestException as e:
    print("Request error:", e)

Request URL: https://api.isthereanydeal.com/games/history/v2?key=6e83d100e0379ecd168d625928bec6a244dd08d4&id=018d937f-0184-7248-8d64-3c723c523111&country=us&shops=61&since=2018-01-01T00%3A00%3A00Z
Price history data: [{'timestamp': '2025-09-01T19:16:32+02:00', 'shop': {'id': 61, 'name': 'Steam'}, 'deal': {'price': {'amount': 8.99, 'amountInt': 899, 'currency': 'USD'}, 'regular': {'amount': 59.99, 'amountInt': 5999, 'currency': 'USD'}, 'cut': 85}}, {'timestamp': '2025-07-27T19:15:28+02:00', 'shop': {'id': 61, 'name': 'Steam'}, 'deal': {'price': {'amount': 59.99, 'amountInt': 5999, 'currency': 'USD'}, 'regular': {'amount': 59.99, 'amountInt': 5999, 'currency': 'USD'}, 'cut': 0}}, {'timestamp': '2025-07-20T19:15:45+02:00', 'shop': {'id': 61, 'name': 'Steam'}, 'deal': {'price': {'amount': 11.99, 'amountInt': 1199, 'currency': 'USD'}, 'regular': {'amount': 59.99, 'amountInt': 5999, 'currency': 'USD'}, 'cut': 80}}, {'timestamp': '2025-07-10T19:33:58+02:00', 'shop': {'id': 61, 'name': 'Steam'

In [96]:
from datetime import datetime, timezone
from typing import List, Dict, Optional

def _parse_ts(ts: str) -> datetime:
    # handles both ...Z and timezone offsets like +02:00; returns UTC
    dt = datetime.fromisoformat(ts.replace("Z", "+00:00"))
    return dt.astimezone(timezone.utc)

def _dedupe_same_timestamp(events: List[Dict]) -> List[Dict]:
    """
    For events sharing the same timestamp, keep the one with the largest cut.
    This avoids double-logs from scraper updates at identical times.
    """
    bucket = {}
    for e in events:
        t = e["timestamp"]
        cut = (e.get("deal") or {}).get("cut") or 0
        if (t not in bucket) or (cut > ((bucket[t].get("deal") or {}).get("cut") or 0)):
            bucket[t] = e
    return [bucket[t] for t in sorted(bucket.keys(), key=_parse_ts)]

def find_first_sale(
    history: List[Dict],
    release_dt: Optional[datetime] = None,
    min_cut: int = 5,
    post_launch_only: bool = True,
) -> Optional[Dict]:
    """
    history: full list from ITAD /games/history/v2 (already filtered to shop=61, country fixed)
    release_dt: UTC datetime (from /games/info/v2). If None, no pre-release filtering is applied.
    min_cut: minimum % discount to consider a sale
    post_launch_only: ignore events strictly before release_dt if True
    returns dict with keys: timestamp (UTC dt), cut, price, regular  OR None if no sale
    """
    if not history:
        return None

    # normalize + sort
    events_sorted = sorted(history, key=lambda e: _parse_ts(e["timestamp"]))
    # dedupe by identical timestamp (keep max cut)
    events_sorted = _dedupe_same_timestamp(events_sorted)

    for e in events_sorted:
        ts = _parse_ts(e["timestamp"])
        if post_launch_only and release_dt is not None and ts < release_dt:
            continue

        deal = e.get("deal") or {}
        cut = deal.get("cut") or 0
        price = (deal.get("price") or {}).get("amount")
        regular = (deal.get("regular") or {}).get("amount")

        # sanity checks for a true sale
        if cut >= min_cut and price is not None and regular is not None and price < regular:
            return {
                "timestamp": ts,
                "cut": int(cut),
                "price": float(price),
                "regular": float(regular),
            }

    return None

def days_between(a: datetime, b: datetime) -> int:
    return (b - a).days

In [98]:
# 1) get release date (UTC) from /games/info/v2
release_iso = info.get("releaseDate")  # e.g., "2025-08-20T00:00:00Z"
release_dt = datetime.fromisoformat(release_iso.replace("Z", "+00:00")).astimezone(timezone.utc)

# 2) get full history list from /games/history/v2 (shop=61, country consistent)
events = history if isinstance(history, list) else history.get("history", [])

# 3) compute label
first_sale = find_first_sale(events, release_dt=release_dt, min_cut=5, post_launch_only=True)

if first_sale:
    label_days = days_between(release_dt, first_sale["timestamp"])
    print({
        "first_sale_date": first_sale["timestamp"].date().isoformat(),
        "first_sale_cut": first_sale["cut"],
        "first_sale_price": first_sale["price"],
        "first_sale_regular": first_sale["regular"],
        "days_to_first_sale": label_days,
    })
else:
    print({"first_sale_date": None, "days_to_first_sale": None, "censored": True})

{'first_sale_date': '2018-11-21', 'first_sale_cut': 33, 'first_sale_price': 40.19, 'first_sale_regular': 59.99, 'days_to_first_sale': 49}


In [99]:
kaggle_data = pd.read_csv("Kaggle datasets/93182_steam_games.csv")
kaggle_data.head()

/var/folders/1f/b537t8w54jj9x40zq_pnqq4c0000gn/T/ipykernel_47894/4040152700.py:1: DtypeWarning: Columns (3,36) have mixed types. Specify dtype option on import or set low_memory=False.
  kaggle_data = pd.read_csv("Kaggle datasets/93182_steam_games.csv")


,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,...,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
0,1424640,余烬,"Oct 3, 2020",20000 - 50000,0,0,3.99,0,'Ashes of war' is an anti war theme adventure ...,['Simplified Chinese'],...,0,0,0,宁夏华夏西部影视城有限公司,宁夏华夏西部影视城有限公司,"Single-player,Family Sharing","Adventure,Casual,Indie,RPG","Sokoban,RPG,Puzzle-Platformer,Exploration,Adve...",https://shared.akamai.steamstatic.com/store_it...,http://video.akamai.steamstatic.com/store_trai...
1,402890,Nyctophilia,"Sep 23, 2015",50000 - 100000,0,0,0.00,0,NYCTOPHILIA Nyctophilia is an 2D psychological...,"['English', 'Russian']",...,0,0,0,Cat In A Jar Games,Cat In A Jar Games,Single-player,"Adventure,Free To Play,Indie","Free to Play,Indie,Adventure,Horror,2D,Pixel G...",https://shared.akamai.steamstatic.com/store_it...,http://video.akamai.steamstatic.com/store_trai...
2,1151740,Prison Princess,"Apr 2, 2020",0 - 20000,0,0,19.99,0,"ABOUT Now nothing more than a phantom, can the...","['English', 'Simplified Chinese', 'Traditional...",...,0,0,0,qureate,qureate,"Single-player,Steam Achievements,Full controll...","Adventure,Indie","Sexual Content,Adventure,Indie,Nudity,Anime,Ma...",https://shared.akamai.steamstatic.com/store_it...,http://video.akamai.steamstatic.com/store_trai...
3,875530,Dead In Time,"Oct 12, 2018",0 - 20000,0,0,7.99,0,Is a hardcore action with a non-trivial level ...,"['English', 'Russian']",...,0,0,0,Zelenov Artem,Zelenov Artem,"Single-player,Full controller support,Family S...","Action,Indie","Action,Indie,Souls-like,Fantasy,Early Access,R...",https://shared.akamai.steamstatic.com/store_it...,http://video.akamai.steamstatic.com/store_trai...
4,1835360,Panacle: Back To Wild,"Mar 11, 2022",0 - 20000,2,0,3.99,0,Panacle: Back to the Wild is a indie card game...,"['English', 'Japanese', 'Simplified Chinese', ...",...,0,0,0,渡鸦游戏,"渡鸦游戏,电钮组","Single-player,Family Sharing","Indie,Strategy,Early Access","Trading Card Game,Turn-Based Strategy,Lore-Ric...",https://shared.akamai.steamstatic.com/store_it...,http://video.akamai.steamstatic.com/store_trai...


In [104]:
kaggle_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93182 entries, 0 to 93181
Data columns (total 39 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   AppID                       93182 non-null  int64  
 1   Name                        93179 non-null  object 
 2   Release date                93182 non-null  object 
 3   Estimated owners            16462 non-null  object 
 4   Peak CCU                    93182 non-null  int64  
 5   Required age                93182 non-null  int64  
 6   Price                       93182 non-null  float64
 7   DLC count                   93182 non-null  int64  
 8   About the game              88392 non-null  object 
 9   Supported languages         93182 non-null  object 
 10  Full audio languages        93182 non-null  object 
 11  Reviews                     10599 non-null  object 
 12  Header image                93182 non-null  object 
 13  Website                     416

In [105]:
kaggle_data['AppID'].nunique()

93182

In [107]:
kaggle_data["Release date"] = pd.to_datetime(kaggle_data["Release date"], format="%b %d, %Y", errors="coerce")

In [108]:
kaggle_data.head()

,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,...,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
0,1424640,余烬,2020-10-03,20000 - 50000,0,0,3.99,0,'Ashes of war' is an anti war theme adventure ...,['Simplified Chinese'],...,0,0,0,宁夏华夏西部影视城有限公司,宁夏华夏西部影视城有限公司,"Single-player,Family Sharing","Adventure,Casual,Indie,RPG","Sokoban,RPG,Puzzle-Platformer,Exploration,Adve...",https://shared.akamai.steamstatic.com/store_it...,http://video.akamai.steamstatic.com/store_trai...
1,402890,Nyctophilia,2015-09-23,50000 - 100000,0,0,0.00,0,NYCTOPHILIA Nyctophilia is an 2D psychological...,"['English', 'Russian']",...,0,0,0,Cat In A Jar Games,Cat In A Jar Games,Single-player,"Adventure,Free To Play,Indie","Free to Play,Indie,Adventure,Horror,2D,Pixel G...",https://shared.akamai.steamstatic.com/store_it...,http://video.akamai.steamstatic.com/store_trai...
2,1151740,Prison Princess,2020-04-02,0 - 20000,0,0,19.99,0,"ABOUT Now nothing more than a phantom, can the...","['English', 'Simplified Chinese', 'Traditional...",...,0,0,0,qureate,qureate,"Single-player,Steam Achievements,Full controll...","Adventure,Indie","Sexual Content,Adventure,Indie,Nudity,Anime,Ma...",https://shared.akamai.steamstatic.com/store_it...,http://video.akamai.steamstatic.com/store_trai...
3,875530,Dead In Time,2018-10-12,0 - 20000,0,0,7.99,0,Is a hardcore action with a non-trivial level ...,"['English', 'Russian']",...,0,0,0,Zelenov Artem,Zelenov Artem,"Single-player,Full controller support,Family S...","Action,Indie","Action,Indie,Souls-like,Fantasy,Early Access,R...",https://shared.akamai.steamstatic.com/store_it...,http://video.akamai.steamstatic.com/store_trai...
4,1835360,Panacle: Back To Wild,2022-03-11,0 - 20000,2,0,3.99,0,Panacle: Back to the Wild is a indie card game...,"['English', 'Japanese', 'Simplified Chinese', ...",...,0,0,0,渡鸦游戏,"渡鸦游戏,电钮组","Single-player,Family Sharing","Indie,Strategy,Early Access","Trading Card Game,Turn-Based Strategy,Lore-Ric...",https://shared.akamai.steamstatic.com/store_it...,http://video.akamai.steamstatic.com/store_trai...


In [116]:
kaggle_data[(kaggle_data['Release date'] > '2021-01-01') & (kaggle_data['Release date'] < '2024-12-31')]

,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,...,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
4,1835360,Panacle: Back To Wild,2022-03-11,0 - 20000,2,0,3.99,0,Panacle: Back to the Wild is a indie card game...,"['English', 'Japanese', 'Simplified Chinese', ...",...,0,0,0,渡鸦游戏,"渡鸦游戏,电钮组","Single-player,Family Sharing","Indie,Strategy,Early Access","Trading Card Game,Turn-Based Strategy,Lore-Ric...",https://shared.akamai.steamstatic.com/store_it...,http://video.akamai.steamstatic.com/store_trai...
7,2604580,THE JUSOU 3,2024-02-20,0 - 20000,1,0,9.99,0,■ The Ultimate Horror Escape Game 'THE JUSOU -...,"['Japanese', 'English', 'Simplified Chinese', ...",...,0,0,0,株式会社Metaware,株式会社Metaware,"Single-player,Family Sharing","Adventure,Casual,Indie","Exploration,Puzzle,Female Protagonist,2D,3D,Ho...",https://shared.akamai.steamstatic.com/store_it...,http://video.akamai.steamstatic.com/store_trai...
12,2261280,WHITE WATER,2023-01-16,0 - 20000,0,0,1.99,0,Sincerely BLACK COFFEE.,['English'],...,0,0,0,Black Coffee,Black Coffee,"Single-player,Family Sharing","Action,Adventure,Indie,RPG","Adventure,RPG,Interactive Fiction,Action-Adven...",https://shared.akamai.steamstatic.com/store_it...,NaN
13,2325490,Minamochi Factory,2023-06-22,0 - 0,0,0,0.00,0,You can be a master of inspection from today! ...,"['English', 'Japanese']",...,0,0,0,ダイスマン,ダイスマン,"Single-player,Steam Achievements,Steam Cloud","Action,Casual,Free To Play,Indie",NaN,https://shared.akamai.steamstatic.com/store_it...,http://video.akamai.steamstatic.com/store_trai...
15,2214650,Rolando Deluxe,2024-04-17,0 - 20000,0,0,9.99,0,Roll into action and lead a lovable band of Ro...,"['English', 'French', 'German', 'Spanish - Spa...",...,0,0,0,HandCircus,HandCircus,"Single-player,Steam Achievements,Full controll...","Action,Casual,Indie","2D Platformer,Puzzle-Platformer,Cartoony,Physi...",https://shared.akamai.steamstatic.com/store_it...,http://video.akamai.steamstatic.com/store_trai...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93176,2452810,"Alien IQ Exam: Human Edition, Phase 1",2023-07-06,NaN,0,0,12.99,0,Are humans worthy? Take up the challenge by so...,['English'],...,0,0,0,Super RR Man Productions,Super RR Man Productions,"Single-player,Family Sharing",Casual,NaN,https://shared.akamai.steamstatic.com/store_it...,http://video.akamai.steamstatic.com/store_trai...
93177,2251030,Mutant Monty (C64/CPC/Spectrum),2023-01-05,NaN,0,0,4.99,0,Originally released in 1984 for home microcomp...,['English'],...,0,0,0,Artic Computing,Pixel Games UK,"Single-player,Partial Controller Support,Steam...",Action,NaN,https://shared.akamai.steamstatic.com/store_it...,http://video.akamai.steamstatic.com/store_trai...
93179,1844230,Malicious ReloadⅡ,2023-09-05,NaN,0,0,5.99,0,★ To ensure that the game you have purchased w...,"['Japanese', 'English', 'Simplified Chinese', ...",...,0,0,0,UNDER HILL,Playmeow,"Single-player,Family Sharing","Action,Adventure,Simulation",NaN,https://shared.akamai.steamstatic.com/store_it...,http://video.akamai.steamstatic.com/store_trai...
93180,2623690,Mutant Frog,2024-01-27,NaN,0,0,0.99,0,As a result of an unknown meteorite hitting an...,['English'],...,0,0,0,Run-O Games,Run-O Games,"Single-player,Family Sharing","Action,Adventure,Casual,Indie",NaN,https://shared.akamai.steamstatic.com/store_it...,http://video.akamai.steamstatic.com/store_trai...


### I am grabbing all the Steam AppIDs from the 2021 upto 2024

In [118]:
filtered = kaggle_data[(kaggle_data['Release date'] > '2021-01-01') &
                       (kaggle_data['Release date'] < '2024-12-31')]

appids = filtered['AppID'].dropna().astype(int).drop_duplicates().tolist()

53126